<a href="https://colab.research.google.com/github/JoshuaHaga/Covid_19_sprint_1_buildwk/blob/master/Covid_19_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Must run cell below to load data ###

In [2]:
!wget 'https://storage.googleapis.com/kaggle-data-sets/555089%2F1416097%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1598147753&Signature=IukcSTm6w5zUlHaMV4oEP54bDH7VN6pQZQ5IXC5lPv0iVDEdglpdZ3muSmSpuCc56KZKg4zErk2hCG5BocMkfIwGNamqjqJtdHJFKdL4LC2jU%2B71KL%2BVFwfpv6q907o7MztOCZ44oC444RnrTDiobim5%2FdcBZAJ2GYLKpV3TdavOVMtWxhnWDs1r0BhwLLlWFINNa5eRfpmSn9n5MDSkBwZGq9nmmNMVjJHRv%2FRTfflREET2aTC8hPBFSQQN57Wk6HBvWzSzWrbxv91qZ0RU8EY%2BocXQ7jjshUxJzNr1%2BAqlmSIEB9azuFGzUw0JeJoXzbLjS59PLj3R0k2TMEUb%2Fg%3D%3D'
!unzip '/content/555089%2F1416097%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1598147753&Signature=IukcSTm6w5zUlHaMV4oEP54bDH7VN6pQZQ5IXC5lPv0iVDEdglpdZ3muSmSpuCc56KZKg4zErk2hCG5BocMkfIwGNamqjqJtdHJ'

The name is too long, 487 chars total.
Trying to shorten...
New name is 555089%2F1416097%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1598147753&Signature=IukcSTm6w5zUlHaMV4oEP54bDH7VN6pQZQ5IXC5lPv0iVDEdglpdZ3muSmSpuCc56KZKg4zErk2hCG5BocMkfIwGNamqjqJtdHJ.
--2020-08-21 08:18:40--  https://storage.googleapis.com/kaggle-data-sets/555089%2F1416097%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1598147753&Signature=IukcSTm6w5zUlHaMV4oEP54bDH7VN6pQZQ5IXC5lPv0iVDEdglpdZ3muSmSpuCc56KZKg4zErk2hCG5BocMkfIwGNamqjqJtdHJFKdL4LC2jU%2B71KL%2BVFwfpv6q907o7MztOCZ44oC444RnrTDiobim5%2FdcBZAJ2GYLKpV3TdavOVMtWxhnWDs1r0BhwLLlWFINNa5eRfpmSn9n5MDSkBwZGq9nmmNMVjJHRv%2FRTfflREET2aTC8hPBFSQQN57Wk6HBvWzSzWrbxv91qZ0RU8EY%2BocXQ7jjshUxJzNr1%2BAqlmSIEB9azuFGzUw0JeJoXzbLjS59PLj3R0k2TMEUb%2Fg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 173.194.216.128, ...
C

##COVID-19 in USA

#Context
Data is obtained from COVID-19 Tracking project and NYTimes. Sincere thanks to them for making it available to the public.

Coronaviruses are a large family of viruses which may cause illness in animals or humans. In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19 - World Health Organization

The number of new cases are increasing day by day around the world. This dataset has information from 50 US states and the District of Columbia at daily level.

The Original Dataset can be found on kaggle by clicking [here](https://www.kaggle.com/sudalairajkumar/covid19-in-usa).

#Index
All datasets, functions, libraries, and variables used throughout the notebook  will be outlined here. 

###Libraries

In [116]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


###Datasets

In [117]:
### These 3 dataframes come from the kaggle dataset 'COVID-19 in USA'
counties = pd.read_csv('/content/us_counties_covid19_daily.csv')
states   = pd.read_csv('/content/us_states_covid19_daily.csv')
us       = pd.read_csv('/content/us_states_covid19_daily.csv')

###Functions

In [118]:
### Checks for null values. 
def null_check(dataframe):
    val = dataframe.isnull().sum()
    val_sum = val.sum()
    if val_sum == 0:
      print('Zero null values.')
    else:
      print(val)

### Finds null counts, shape, describes and previews data set. 
def explore(dataframe):
  print(null_check(dataframe))
  print('Shape: ', dataframe.shape)
  print(dataframe.describe())
  return dataframe.head()

### Creates a list from column names
def col_list(dataframe):
  val_counts = dataframe.value_counts()
  val_as_pd  = pd.DataFrame(val_counts).T
  val_list   = val_as_pd.columns.tolist()
  val_pd     = pd.DataFrame(val_list, columns=['county'])
  return val_pd

### Calculates daily averages of new cases for counties. Outputs as pandas df
def cnty_daily_avg(dataframe, countyname):
  county = dataframe[dataframe['county']==countyname]
  mean   = county['cases'].mean()
  avg    = mean/len(county)
  dat    = {'county_name':[countyname],'avg_daily':[avg]}
  df     = pd.DataFrame(dat)
  return df

###Applies county daily average function across dataframe from a list. 
def daily_df(dataframe, countylist,):
  data_list = []
  for i in countylist:
    data_list.append(cnty_daily_avg(dataframe, i)) 
  return pd.DataFrame(data_list, columns=['county','daily_avg'])

###Creates a dataframe of daily averages for any state. 
def prep_counties(df, state):
  state = df[df['state']==state]
  county_names = col_list(state['county']) 
  ste = state[state['county']=='Baltimore']
  state_daily_avg = daily_df(state, county_names,)
  return state_daily_avg

###Variables

In [81]:

################################################################################
###########################<<---- Maryland ---->>###############################
################################################################################
### md_counties                  #< - - - - - - - - - - - All Counties in MD ###
### md_counties_vc               #< - - - - - Value counts of counties in MD ###
### md_county_names              #< - - - - - - - - - List of counties in MD ###
### baltimore_daily_average      #< - - - - Average of new cases daily in MD ###
### md_daily_avg                 #< - - - This is a dataframe of the average ###
###                              #      number of new cases per day for each ###
###                              #      county in the state of MD.           ###
###

# Data Exploration

### Introduction:
  Here I will. be doing some basic data explorations of the sets listed in the index. I will clean data here and begin to draw conclusions to be processed later. Findings will be collected and expounded upon at the end of this section. 

##Counties 
Here I am going to explore the counties data set and see what all I can find. Since I live in MD, I am going to first take a look at the number of positive cases since the start of data collection

In [120]:
### Doing some initial exploration in the counties dataset ###
explore(counties)

date         0
county       0
state        0
fips      4250
cases        0
deaths       0
dtype: int64
None
Shape:  (427831, 6)
                fips          cases         deaths
count  423581.000000  427831.000000  427831.000000
mean    31028.521591     706.072795      31.361608
std     16213.346592    5023.235684     383.072950
min      1001.000000       0.000000       0.000000
25%     18147.000000      10.000000       0.000000
50%     29167.000000      48.000000       1.000000
75%     46075.000000     234.000000       6.000000
max     78030.000000  233422.000000   23602.000000


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [121]:
#counties['state'].mean()

In [122]:
md_counties = counties[counties['state']=='Maryland']
md_counties

,date,county,state,fips,cases,deaths
555,2020-03-05,Montgomery,Maryland,24031.0,3,0
615,2020-03-06,Montgomery,Maryland,24031.0,3,0
700,2020-03-07,Montgomery,Maryland,24031.0,3,0
801,2020-03-08,Harford,Maryland,24025.0,1,0
802,2020-03-08,Montgomery,Maryland,24031.0,4,0
...,...,...,...,...,...,...
425824,2020-08-12,Talbot,Maryland,24041.0,404,4
425825,2020-08-12,Unknown,Maryland,NaN,0,14
425826,2020-08-12,Washington,Maryland,24043.0,1041,31
425827,2020-08-12,Wicomico,Maryland,24045.0,1361,45


In [123]:
### Checking fo null data 
null_check(md_counties)

date        0
county      0
state       0
fips      122
cases       0
deaths      0
dtype: int64


In [124]:
##### Counts of counties shows pretty even sampling across counties. 
md_counties_vc = md_counties['county'].value_counts()
md_counties_vc

Montgomery         161
Harford            158
Prince George's    157
Baltimore          155
Charles            153
Anne Arundel       153
Carroll            152
Baltimore city     151
Howard             151
Frederick          150
Talbot             150
Calvert            147
Wicomico           147
Worcester          146
Caroline           145
Washington         145
Somerset           145
Cecil              144
St. Mary's         144
Garrett            143
Queen Anne's       143
Kent               141
Dorchester         134
Allegany           133
Unknown            122
Name: county, dtype: int64

In [125]:
#Creating a list of county names in md
mdcounty_list = pd.DataFrame(md_counties_vc)
mdcounty_list = mdcounty_list.T
mdcounty_list

,Montgomery,Harford,Prince George's,Baltimore,Charles,Anne Arundel,Carroll,Baltimore city,Howard,Frederick,Talbot,Calvert,Wicomico,Worcester,Caroline,Washington,Somerset,Cecil,St. Mary's,Garrett,Queen Anne's,Kent,Dorchester,Allegany,Unknown
county,161,158,157,155,153,153,152,151,151,150,150,147,147,146,145,145,145,144,144,143,143,141,134,133,122


In [126]:
md_county_names2 = mdcounty_list.columns.tolist()
md_county_names2

['Montgomery',
 'Harford',
 "Prince George's",
 'Baltimore',
 'Charles',
 'Anne Arundel',
 'Carroll',
 'Baltimore city',
 'Howard',
 'Frederick',
 'Talbot',
 'Calvert',
 'Wicomico',
 'Worcester',
 'Caroline',
 'Washington',
 'Somerset',
 'Cecil',
 "St. Mary's",
 'Garrett',
 "Queen Anne's",
 'Kent',
 'Dorchester',
 'Allegany',
 'Unknown']

In [127]:
#The whole process as a function to speed up later states. 
def col_list(dataframe):
  val_counts = dataframe.value_counts()
  val_as_pd  = pd.DataFrame(val_counts).T
  val_list   = val_as_pd.columns.tolist()
  val_pd     = pd.DataFrame(val_list, columns=['county'])
  return val_pd

In [128]:
md_county_names = col_list(md_counties['county']) ###<<< - - - - Testing listing function
md_county_names

,county
0,Montgomery
1,Harford
2,Prince George's
3,Baltimore
4,Charles
5,Anne Arundel
6,Carroll
7,Baltimore city
8,Howard
9,Frederick


In [129]:
#Finding average cases per county during the entire time of recording
#Starting with Baltimore

In [130]:
baltimore = md_counties[md_counties['county']=='Baltimore']
baltimore

,date,county,state,fips,cases,deaths
1249,2020-03-11,Baltimore,Maryland,24005.0,1,0
1467,2020-03-12,Baltimore,Maryland,24005.0,1,0
1745,2020-03-13,Baltimore,Maryland,24005.0,2,0
2085,2020-03-14,Baltimore,Maryland,24005.0,3,0
2484,2020-03-15,Baltimore,Maryland,24005.0,3,0
...,...,...,...,...,...,...
412921,2020-08-08,Baltimore,Maryland,24005.0,13082,561
416141,2020-08-09,Baltimore,Maryland,24005.0,13279,563
419362,2020-08-10,Baltimore,Maryland,24005.0,13422,563
422584,2020-08-11,Baltimore,Maryland,24005.0,13396,566


In [131]:
balt_mean = baltimore['cases'].mean()

In [132]:
baltimore_daily_average = balt_mean/len(baltimore)
baltimore_daily_average

35.56853277835588

In [150]:
### Create function to do this faster for later 

def cnty_daily_avg(dataframe, countyname):
  county = dataframe[dataframe['county']==countyname]
  mean   = county['cases'].mean()
  avg    = mean/len(county)
  dat    = [countyname,avg]
  return dat


In [151]:
harf_avg = cnty_daily_avg(md_counties, 'Harford')
harf_avg

['Harford', 4.924651498157347]

In [152]:
balt_avg = cnty_daily_avg(md_counties, 'Baltimore') #< - - - thank god I dont have to do this hundreds of times manually
balt_avg

['Baltimore', 35.56853277835588]

In [153]:
#Apply function to all counties and create a dataframe
data_list = []

In [154]:
def daily_df(dataframe, countylist,):
  data_list = []
  for i in countylist:
    data_list.append(cnty_daily_avg(dataframe, i)) 
  return data_list

In [157]:
md_daily_avg = daily_df(md_counties, md_county_names)
md_daily_avg.head()

ZeroDivisionError: ignored

In [138]:
###Doing everything we did above with one script! Boo Yah!
def prep_counties(df, state):
  state = df[df['state']==state]
  county_names = col_list(state['county']) 
  ste = state[state['county']=='Baltimore']
  state_daily_avg = daily_df(state, county_names,)
  return state_daily_avg

In [161]:
virginia = prep_counties(counties, 'Virginia')
virginia

ZeroDivisionError: ignored

In [160]:
#Lets get a list of states we have data on and apply this to all of our data. 
states = col_list(counties['state'])
states

,county
0,Texas
1,Georgia
2,Virginia
3,Kentucky
4,Missouri
5,North Carolina
6,Illinois
7,Tennessee
8,Iowa
9,Indiana


In [159]:
def counties_daily_avg(dataframe, state_list):
  data_df = []
  for i in state_list :
    data_df.append(prep_counties(dataframe, i)) 
  return data_df

In [158]:
counties_daily_avg(counties, states)

ZeroDivisionError: ignored

##States

In [ ]:
explore(states)

##US


In [ ]:
explore(us)